# Delay Modelling with Random Forests

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from local_nbutils import CFG

In [ ]:
df = pd.read_pickle(CFG["PROCESSED_DATA_PATH"])

In [ ]:
df2 = pd.get_dummies(
    df, columns=["DATOP_day"], prefix="day", drop_first=True, dtype=int
)
df2 = pd.get_dummies(
    df2, columns=["DATOP_year"], prefix="yr", drop_first=True, dtype=int
)
df2 = pd.get_dummies(
    df2, columns=["DATOP_month"], prefix="mon", drop_first=True, dtype=int
)
df2 = pd.get_dummies(df2, columns=["DEPSTN"], prefix="dep", drop_first=True, dtype=int)
df2 = pd.get_dummies(df2, columns=["ARRSTN"], prefix="arr", drop_first=True, dtype=int)
df2 = pd.get_dummies(df2, columns=["AC"], prefix="ac", drop_first=True, dtype=int)
df2 = pd.get_dummies(
    df2, columns=["STD_period"], prefix="std", drop_first=True, dtype=int
)
# df2 = pd.get_dummies(
#     df2, columns=["STA_period"], prefix="sta", drop_first=True, dtype=int
# )

In [ ]:
y = df2.target
X = df2.drop("target", axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=CFG["RSEED"]
)

In [ ]:
prefixes = ["day_", "yr_", "mt_", "ac_", "dep_", "arr_", "std_", "sta_"]

# Collect columns that match those prefixes
feature_cols = [
    col for col in df2.columns if any(col.startswith(p) for p in prefixes)
] + ["flight_time"]

x0 = X_train[feature_cols]
x1 = X_test[feature_cols]


model = LinearRegression()
# model = KNeighborsRegressor(n_neighbors=5)

model.fit(x0, y_train)
y_pred_test = model.predict(x1)

print(np.sqrt(mean_squared_error(y_test, y_pred_test)))
print(r2_score(y_test, y_pred_test))

In [ ]:
# Define the model
model = RandomForestRegressor(random_state=42)

# Define the hyperparameter grid
param_grid = {
    "n_estimators": [100],
    # "max_depth": [20, 30],
    # "min_samples_split": [5, 10],
    # "min_samples_leaf": [2, 5],
    # "max_features": ["auto", "sqrt"],
}

# Create GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring="neg_mean_squared_error",  # or use 'r2' if you prefer
    n_jobs=-1,  # Use all available cores
    verbose=2,
)

# Fit the grid search to the data
grid_search.fit(x0, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predict on test data
y_pred_test = best_model.predict(x1)

# Evaluate
print("Best Parameters:", grid_search.best_params_)
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))
print("R²:", r2_score(y_test, y_pred_test))